<a href="https://colab.research.google.com/github/vilsonrodrigues/COVID19/blob/master/Fraterno20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libs

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

###Leitura de Base de Dados

CSV de leitos de internação por Municípios, obtivo no DataSUS

In [2]:
leitos_internacao = pd.read_csv("https://raw.githubusercontent.com/vilsonrodrigues/COVID19/master/leitos_por_cidade.csv", sep = ";", encoding = "iso-8859-1")
leitos_internacao.rename(columns = {"municipio":"city"}, inplace = True)
leitos_internacao.head(2)

,city,quantidade
0,110001 Alta Floresta D'Oeste,49
1,110037 Alto Alegre dos Parecis,16


Base de dados do Brasil IO, informando dados por município

In [3]:
data_brasil = pd.read_csv('https://data.brasil.io/dataset/covid19/caso.csv.gz', compression='gzip', header=0, quotechar='"', error_bad_lines=False)
data_brasil.rename(columns={"city_ibge_code":"codigo_ibge","state":"sigla_state"},inplace = True)
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,codigo_ibge,confirmed_per_100k_inhabitants,death_rate
0,2020-03-28,RS,NaN,state,200,2.0,16,True,11377239.0,43.0,1.75790,0.0100
1,2020-03-27,RS,NaN,state,197,2.0,15,False,11377239.0,43.0,1.73153,0.0102


In [4]:
data_brasil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2446 entries, 0 to 2445
Data columns (total 12 columns):
date                              2446 non-null object
sigla_state                       2446 non-null object
city                              2056 non-null object
place_type                        2446 non-null object
confirmed                         2446 non-null int64
deaths                            2109 non-null float64
order_for_place                   2446 non-null int64
is_last                           2446 non-null bool
estimated_population_2019         2367 non-null float64
codigo_ibge                       2367 non-null float64
confirmed_per_100k_inhabitants    2339 non-null float64
death_rate                        133 non-null float64
dtypes: bool(1), float64(5), int64(2), object(4)
memory usage: 212.7+ KB


CSV com dados de longitude e latitude dos municípios brasileiros

In [5]:
city_lat_long =  pd.read_csv("https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/municipios.csv")
city_lat_long.drop(columns = ["capital"], inplace = True)
city_lat_long.rename(columns = {"nome":"city"},inplace = True)
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31


Base do IBGE, contendo código do município, nome do estado, do município, e o código do estado

In [6]:
data_ibge = pd.read_excel("https://github.com/vilsonrodrigues/COVID19/blob/master/RELATORIO_DTB_BRASIL_MUNICIPIO.xls?raw=true")
data_ibge.drop(columns = ["Mesorregião Geográfica","Nome_Mesorregião","Microrregião Geográfica","Nome_Microrregião","Município"], inplace = True)
data_ibge.rename(columns = {"Nome_Município":"city","Nome_UF":"state","UF":"codigo_uf","Código Município Completo":"codigo_ibge"},inplace = True)
data_ibge.head(2)

,codigo_uf,state,codigo_ibge,city
0,11,Rondônia,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1100379,Alto Alegre dos Parecis


###Manipulação das bases

Juntando a tabela da localização geográfica com a dos dados do ibge

In [7]:
city_lat_long = city_lat_long.merge(data_ibge, on = ["city","codigo_ibge","codigo_uf"],sort = "city")
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Merge de tabelas de caso e localização geográfica

In [8]:
data_brasil = data_brasil.merge(city_lat_long, on = ["city","codigo_ibge"],sort = "date",how="outer")
data_brasil.dropna(subset=["date"],inplace=True)
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
60,2020-03-29,BA,Alagoinhas,city,1.0,0.0,4.0,True,151596.0,2900702.0,0.65965,NaN,-12.1335,-38.4208,29.0,Bahia
61,2020-03-28,BA,Alagoinhas,city,1.0,NaN,3.0,False,151596.0,2900702.0,0.65965,NaN,-12.1335,-38.4208,29.0,Bahia


In [9]:
data_brasil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2446 entries, 60 to 7694
Data columns (total 16 columns):
date                              2446 non-null object
sigla_state                       2446 non-null object
city                              2056 non-null object
place_type                        2446 non-null object
confirmed                         2446 non-null float64
deaths                            2109 non-null float64
order_for_place                   2446 non-null float64
is_last                           2446 non-null object
estimated_population_2019         2367 non-null float64
codigo_ibge                       2367 non-null float64
confirmed_per_100k_inhabitants    2339 non-null float64
death_rate                        133 non-null float64
latitude                          1977 non-null float64
longitude                         1977 non-null float64
codigo_uf                         1977 non-null float64
state                             1977 non-null object
dty

Retirando NaN's e passando o codigo do ibge pra inteiro

In [11]:
data_brasil.replace(np.nan, 0, inplace = True)
data_brasil["codigo_ibge"] = data_brasil["codigo_ibge"].astype("int64")
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
60,2020-03-29,BA,Alagoinhas,city,1.0,0.0,4.0,True,151596.0,2900702,0.65965,0.0,-12.1335,-38.4208,29.0,Bahia
61,2020-03-28,BA,Alagoinhas,city,1.0,0.0,3.0,False,151596.0,2900702,0.65965,0.0,-12.1335,-38.4208,29.0,Bahia


Usando Split para retirar o código do município

In [12]:
new = leitos_internacao["city"].str.split(" ", n = 1, expand = True)
leitos_internacao["city"] = new[1]
leitos_internacao["codigo_ibge"] = new[0] #Dev sumiu com o ultimo digito de todos os codigos
leitos_internacao.head(2)

,city,quantidade,codigo_ibge
0,Alta Floresta D'Oeste,49,110001
1,Alto Alegre dos Parecis,16,110037


Farei merge, mas pra isso irei fazer uma cópia dos de localização, porque terei que tirar o último dígito de todos os dados pra conseguir juntar com os dos leitos que tem o último faltando

In [13]:
city_lat_long["codigo_ibge"] = city_lat_long["codigo_ibge"].map(lambda x: str(x)[:-1])
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,520005,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,310010,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


In [14]:
leitos_internacao = leitos_internacao.merge(city_lat_long, on = ["city","codigo_ibge"], sort = "city", how = "outer")
leitos_internacao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5603 entries, 0 to 5602
Data columns (total 7 columns):
city           5603 non-null object
quantidade     5596 non-null object
codigo_ibge    5603 non-null object
latitude       5543 non-null float64
longitude      5543 non-null float64
codigo_uf      5543 non-null float64
state          5543 non-null object
dtypes: float64(3), object(4)
memory usage: 350.2+ KB


In [15]:
data_brasil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2446 entries, 60 to 7694
Data columns (total 16 columns):
date                              2446 non-null object
sigla_state                       2446 non-null object
city                              2446 non-null object
place_type                        2446 non-null object
confirmed                         2446 non-null float64
deaths                            2446 non-null float64
order_for_place                   2446 non-null float64
is_last                           2446 non-null bool
estimated_population_2019         2446 non-null float64
codigo_ibge                       2446 non-null int64
confirmed_per_100k_inhabitants    2446 non-null float64
death_rate                        2446 non-null float64
latitude                          2446 non-null float64
longitude                         2446 non-null float64
codigo_uf                         2446 non-null float64
state                             2446 non-null object
dtypes

Dataset com os ultimos boletins

In [16]:
data_brasil_last_boletim = data_brasil.loc[data_brasil["is_last"] == True,:]
data_brasil_last_boletim.head(3)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
60,2020-03-29,BA,Alagoinhas,city,1.0,0.0,4.0,True,151596.0,2900702,0.65965,0.0,-12.13350,-38.4208,29.0,Bahia
145,2020-03-28,RS,Alvorada,city,2.0,0.0,10.0,True,210305.0,4300604,0.95100,0.0,-29.99140,-51.0809,43.0,Rio Grande do Sul
194,2020-03-28,PA,Ananindeua,city,4.0,0.0,4.0,True,530598.0,1500800,0.75387,0.0,-1.36391,-48.3743,15.0,Pará


Retirando Nulos

In [0]:
leitos_internacao.replace("-", 0, inplace = True)

###Dados Tratados

In [18]:
data_brasil.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
60,2020-03-29,BA,Alagoinhas,city,1.0,0.0,4.0,True,151596.0,2900702,0.65965,0.0,-12.1335,-38.4208,29.0,Bahia
61,2020-03-28,BA,Alagoinhas,city,1.0,0.0,3.0,False,151596.0,2900702,0.65965,0.0,-12.1335,-38.4208,29.0,Bahia


In [20]:
data_brasil_last_boletim.head(2)

,date,sigla_state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,codigo_ibge,confirmed_per_100k_inhabitants,death_rate,latitude,longitude,codigo_uf,state
60,2020-03-29,BA,Alagoinhas,city,1.0,0.0,4.0,True,151596.0,2900702,0.65965,0.0,-12.1335,-38.4208,29.0,Bahia
145,2020-03-28,RS,Alvorada,city,2.0,0.0,10.0,True,210305.0,4300604,0.95100,0.0,-29.9914,-51.0809,43.0,Rio Grande do Sul


In [19]:
leitos_internacao.head(2)

,city,quantidade,codigo_ibge,latitude,longitude,codigo_uf,state
0,Abadia de Goiás,0,520005,-16.7573,-49.4412,52.0,Goiás
1,Abadia dos Dourados,17,310010,-18.4831,-47.3916,31.0,Minas Gerais


In [21]:
leitos_internacao.loc[leitos_internacao["city"].str.contains("Pedro Avelino"),:]

,city,quantidade,codigo_ibge,latitude,longitude,codigo_uf,state
3702,Pedro Avelino,0,240970,-5.5161,-36.3867,24.0,Rio Grande do Norte


In [22]:
data_brasil.columns

Index(['date', 'sigla_state', 'city', 'place_type', 'confirmed', 'deaths',
       'order_for_place', 'is_last', 'estimated_population_2019',
       'codigo_ibge', 'confirmed_per_100k_inhabitants', 'death_rate',
       'latitude', 'longitude', 'codigo_uf', 'state'],
      dtype='object')

In [23]:
leitos_internacao.columns

Index(['city', 'quantidade', 'codigo_ibge', 'latitude', 'longitude',
       'codigo_uf', 'state'],
      dtype='object')

Download dos CSV's

In [0]:
#data_brasil_last_boletim.to_csv("data_brasil_last_boletim.csv")

In [0]:
#data_brasil.to_csv("data_brasil.csv")

In [0]:
#leitos_internacao.to_csv("leitos_internacao.csv")

###Montagem de Flask API

In [0]:
import pandas as pd
import os
from flask import Flask, request, render_template, make_response

app = Flask(__name__)

@app.route('/')
def display_gui():
    return render_template('template.html')#html a ser renderizado

@app.route('/leitos_internacao', methods=['POST'])
def leitos_internacao():
  estado = request.form['estado_leitos_interncao']
	municipio = request.form['municipio_leitos_interncao']

  leitos_internacao = pd.read_csv()#de onde vou puxar a base de dados
	quantidade = leitos_internacao.loc[(leitos_internacao["city"].str.contains(municipio)) & (leitos_internacao["state"].str.contains(estado)),"quantidade"]

	return render_template('template.html',quantidade)

if __name__ == "__main__": #Verificar
        port = int(os.environ.get('PORT', 5500))
        app.run(host='0.0.0.0', port=port)

In [0]:
import pandas as pd
import os
from flask import Flask, request, render_template, make_response

app = Flask(__name__)

@app.route('/')
def display_gui():
    return render_template('template.html')#html a ser renderizado

@app.route('/leitos_internacao', methods=['POST'])
def leitos_internacao():
  estado = request.form['estado_leitos_interncao']
	municipio = request.form['municipio_leitos_interncao']

  data_brasil = pd.read_csv()#de onde vou puxar a base de dados
	quantidade = leitos_internacao.loc[(leitos_internacao["city"].str.contains(municipio)) & (leitos_internacao["state"].str.contains(estado)),"quantidade"]

	return render_template('template.html',quantidade)

if __name__ == "__main__": #Verificar
        port = int(os.environ.get('PORT', 5500))
        app.run(host='0.0.0.0', port=port)

In [0]:
import pandas as pd
import os
from flask import Flask, request, render_template, make_response

app = Flask(__name__)

@app.route('/')
def display_gui():
    return render_template('template.html')#html a ser renderizado

@app.route('/leitos_internacao', methods=['POST'])
def leitos_internacao():
  estado = request.form['estado_leitos_interncao']
	municipio = request.form['municipio_leitos_interncao']

  leitos_internacao = pd.read_csv()#de onde vou puxar a base de dados
	quantidade = leitos_internacao.loc[(leitos_internacao["city"].str.contains(municipio)) & (leitos_internacao["state"].str.contains(estado)),"quantidade"]

	return render_template('template.html',quantidade)

if __name__ == "__main__": #Verificar
        port = int(os.environ.get('PORT', 5500))
        app.run(host='0.0.0.0', port=port)